In [4]:
import tensorflow as tf
from tensorflow.keras import layers, models
import pandas as pd
import numpy as np
import cv2
import os
import math
from sklearn.model_selection import train_test_split

train_img_dir   = "Nutrition5K/train/color"
train_depth_dir = "Nutrition5K/train/depth_raw"
test_img_dir    = "Nutrition5K/test/color"
test_depth_dir  = "Nutrition5K/test/depth_raw"
label_file      = "Nutrition5K/nutrition5k_train.csv"

epochs = 70
image_size = (224, 224)
batch_size = 64
AUTOTUNE = tf.data.AUTOTUNE

def check_file(path):
    return os.path.exists(path) and os.path.getsize(path) > 0

label_df = pd.read_csv(label_file)
label_df["img_path"]   = label_df["ID"].apply(lambda x: os.path.join(train_img_dir, str(x), "rgb.png"))
label_df["depth_path"] = label_df["ID"].apply(lambda x: os.path.join(train_depth_dir, str(x), "depth_raw.png"))
## label_df = label_df[(label_df["Value"] < 2500) & (label_df["Value"] > 0)]
label_df = label_df[label_df["img_path"].apply(check_file) & label_df["depth_path"].apply(check_file)].reset_index(drop=True)
#print(f Valid samples after filtering: {len(label_df)}")

CAMERA_DISTANCE_CM = 35.9
PIXEL_AREA_CM2 = 5.957e-3

def estimate_food_volume(depth_path):
    depth = cv2.imread(depth_path, cv2.IMREAD_UNCHANGED).astype(np.float32)
    if depth is None or depth.size == 0:
        return 0.0
    hist, bins = np.histogram(depth.flatten(), bins=100)
    plate_depth = bins[np.argmax(hist)]
    mask = (plate_depth - depth) > 10
    food_depth = np.clip((plate_depth - depth), 0, None)
    volume_per_pixel = food_depth * PIXEL_AREA_CM2
    total_volume = np.sum(volume_per_pixel * mask)
    return total_volume / 1000.0

label_df["Volume"] = label_df["depth_path"].apply(estimate_food_volume).astype("float32")
train_df, val_df = train_test_split(label_df, test_size=0.15, random_state=42)


def preprocess_rgb_depth(img_path, depth_path, label, volume, image_size=(224,224)):
    rgb = tf.io.read_file(img_path)
    rgb = tf.image.decode_png(rgb, channels=3)
    rgb = tf.image.resize(rgb, image_size)
    rgb = tf.cast(rgb, tf.float32) / 255.0

    depth = tf.io.read_file(depth_path)
    depth = tf.image.decode_png(depth, channels=1)
    depth = tf.image.resize(depth, image_size)
    depth = tf.cast(depth, tf.float32) / 255.0

    fused = tf.concat([rgb, depth], axis=-1)
    return {"fused_input": fused, "volume_input": tf.expand_dims(volume, -1)}, label

def build_dataset(df, batch_size=32, augment=False):
    paths_img = df["img_path"].values
    paths_depth = df["depth_path"].values
    labels = df["Value"].astype("float32").values
    volumes = df["Volume"].astype("float32").values
    ds = tf.data.Dataset.from_tensor_slices((paths_img, paths_depth, labels, volumes))
    ds = ds.map(lambda p1, p2, y, v: preprocess_rgb_depth(p1, p2, y, v, image_size),
                num_parallel_calls=AUTOTUNE)
    if augment:
        def aug_fn(x, y):
            if tf.random.uniform(()) > 0.5:
                x["fused_input"] = tf.image.flip_left_right(x["fused_input"])
            return x, y
        ds = ds.map(aug_fn, num_parallel_calls=AUTOTUNE)
        ds = ds.shuffle(1000)
    return ds.batch(batch_size).prefetch(AUTOTUNE)

train_ds = build_dataset(train_df, batch_size=batch_size, augment=True)
val_ds   = build_dataset(val_df, batch_size=batch_size, augment=False)



def inception_block(x, f1=32, f3r=32, f3=48, f5r=16, f5=32, fpool=32):
    b1 = layers.Conv2D(f1, 1, activation='relu', padding='same')(x)
    b2 = layers.Conv2D(f3r, 1, activation='relu', padding='same')(x)
    b2 = layers.Conv2D(f3, 3, activation='relu', padding='same')(b2)
    b3 = layers.Conv2D(f5r, 1, activation='relu', padding='same')(x)
    b3 = layers.Conv2D(f5, 5, activation='relu', padding='same')(b3)
    b4 = layers.MaxPooling2D(3, strides=1, padding='same')(x)
    b4 = layers.Conv2D(fpool, 1, activation='relu', padding='same')(b4)
    out = layers.Concatenate()([b1,b2,b3,b4])
    out = layers.BatchNormalization()(out)
    return layers.ReLU()(out)

def build_unified_inception(input_shape=(224,224,4), dropout=0.3):
    fused_input = layers.Input(shape=input_shape, name = "fused_input")

    x = layers.Conv2D(32, 3, strides=2, padding='same', activation='relu')(fused_input)
    x = layers.Conv2D(32, 3, activation='relu', padding='same')(x)
    x = layers.MaxPooling2D(3, strides=2, padding='same')(x)
    x = inception_block(x, 32, 32, 48, 16, 32, 32)
    x = layers.MaxPooling2D(2)(x)
    x = inception_block(x, 64, 48, 64, 24, 48, 48)
    x = layers.GlobalAveragePooling2D()(x)
    x = layers.Dense(128, activation='relu')(x)
    x = layers.Dropout(dropout)(x)

    # Late Fusion for Volume
    vol_input = layers.Input(shape=(1,), name="volume_input")
    v = layers.Dense(16, activation='relu')(vol_input)

    # Fusion
    fused = layers.Concatenate()([x, v])
    fused = layers.Dense(64, activation='relu')(fused)
    fused = layers.Dropout(dropout)(fused)
    output = layers.Dense(1, activation='relu', name="total_value")(fused)
    model = models.Model(inputs=[fused_input, vol_input], outputs=output)
    return model

model = build_unified_inception()
model.summary()

optimizer = tf.keras.optimizers.Adam(learning_rate=1e-4, weight_decay=1e-4)
model.compile(optimizer=optimizer, loss="mse", metrics=["mae"])

callbacks = [
    tf.keras.callbacks.ModelCheckpoint("best_inception.keras", save_best_only=True, monitor="val_loss"),
]

history = model.fit(train_ds, validation_data=val_ds, epochs=epochs, callbacks=callbacks)


Model: "functional_3"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ fused_input         │ (None, 224, 224,  │          0 │ -                 │
│ (InputLayer)        │ 4)                │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_42 (Conv2D)  │ (None, 112, 112,  │      1,184 │ fused_input[0][0] │
│                     │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_43 (Conv2D)  │ (None, 112, 112,  │      9,248 │ conv2d_42[0][0]   │
│                     │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ max_pooling2d_12    │ (None, 56, 56,    │          0 │ conv2d_43[0][0]   │
│ (MaxPooling2D)      │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_45 (Conv2D)  │ (None, 56, 56,    │      1,056 │ max_pooling2d_12… │
│                     │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_47 (Conv2D)  │ (None, 56, 56,    │        528 │ max_pooling2d_12… │
│                     │ 16)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ max_pooling2d_13    │ (None, 56, 56,    │          0 │ max_pooling2d_12… │
│ (MaxPooling2D)      │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_44 (Conv2D)  │ (None, 56, 56,    │      1,056 │ max_pooling2d_12… │
│                     │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_46 (Conv2D)  │ (None, 56, 56,    │     13,872 │ conv2d_45[0][0]   │
│                     │ 48)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_48 (Conv2D)  │ (None, 56, 56,    │     12,832 │ conv2d_47[0][0]   │
│                     │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_49 (Conv2D)  │ (None, 56, 56,    │      1,056 │ max_pooling2d_13… │
│                     │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ concatenate_9       │ (None, 56, 56,    │          0 │ conv2d_44[0][0],  │
│ (Concatenate)       │ 144)              │            │ conv2d_46[0][0],  │
│                     │                   │            │ conv2d_48[0][0],  │
│                     │                   │            │ conv2d_49[0][0]   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ batch_normalizatio… │ (None, 56, 56,    │        576 │ concatenate_9[0]… │
│ (BatchNormalizatio… │ 144)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ re_lu_6 (ReLU)      │ (None, 56, 56,    │          0 │ batch_normalizat… │
│                     │ 144)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ max_pooling2d_14    │ (None, 28, 28,    │          0 │ re_lu_6[0][0]     │
│ (MaxPooling2D)      │ 144)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_51 (Conv2D)  │ (None, 28, 28,    │      6,960 │ max_pooling2d_14… │
│                     │ 48)               │            │                 

 Total params: 163,721 (639.54 KB)

 Trainable params: 162,985 (636.66 KB)

 Non-trainable params: 736 (2.88 KB)

Epoch 1/70
44/44 ━━━━━━━━━━━━━━━━━━━━ 31s 653ms/step - loss: 92773.3125 - mae: 218.1578 - val_loss: 83275.3438 - val_mae: 205.0494
Epoch 2/70
44/44 ━━━━━━━━━━━━━━━━━━━━ 32s 707ms/step - loss: 77050.8828 - mae: 200.9547 - val_loss: 68777.5547 - val_mae: 195.9425
Epoch 3/70
44/44 ━━━━━━━━━━━━━━━━━━━━ 33s 727ms/step - loss: 72017.6797 - mae: 198.9825 - val_loss: 67119.1016 - val_mae: 197.1780
Epoch 4/70
44/44 ━━━━━━━━━━━━━━━━━━━━ 35s 756ms/step - loss: 71925.2109 - mae: 199.7707 - val_loss: 66770.6406 - val_mae: 197.9859
Epoch 5/70
44/44 ━━━━━━━━━━━━━━━━━━━━ 34s 748ms/step - loss: 66691.4766 - mae: 193.1613 - val_loss: 66936.1641 - val_mae: 197.1690
Epoch 6/70
44/44 ━━━━━━━━━━━━━━━━━━━━ 34s 732ms/step - loss: 64952.0781 - mae: 188.0517 - val_loss: 66549.9844 - val_mae: 196.4732
Epoch 7/70
44/44 ━━━━━━━━━━━━━━━━━━━━ 34s 747ms/step - loss: 59941.5312 - mae: 181.8296 - val_loss: 66856.0078 - val_mae: 194.8716
Epoch 8/70
44/44 ━━━━━━━━━━━━━━━━━━━━ 34s 750ms/step - loss: 53089.7930 - mae: 168.

In [6]:
def build_test_dataset(df, batch_size=32):
    paths_img = df["img_path"].values
    paths_depth = df["depth_path"].values
    volumes = df["VolumeEstimate"].astype("float32").values

    ds = tf.data.Dataset.from_tensor_slices((paths_img, paths_depth, volumes))

    def _load(p_img, p_depth, vol):
        rgb = tf.io.read_file(p_img)
        rgb = tf.image.decode_png(rgb, channels=3)
        rgb = tf.image.resize(rgb, image_size)
        rgb = tf.cast(rgb, tf.float32) / 255.0

        depth = tf.io.read_file(p_depth)
        depth = tf.image.decode_png(depth, channels=1)
        depth = tf.image.resize(depth, image_size)
        depth = tf.cast(depth, tf.float32) / 255.0

        fused = tf.concat([rgb, depth], axis=-1)
        return {"fused_input": fused, "volume_input": tf.expand_dims(vol, -1)}

    ds = ds.map(_load, num_parallel_calls=AUTOTUNE)
    return ds.batch(batch_size).prefetch(AUTOTUNE)

    
test_ids = sorted([
    d for d in os.listdir(test_img_dir)
    if os.path.isdir(os.path.join(test_img_dir, d))
])

test_df = pd.DataFrame(test_ids, columns=["ID"])
test_df["img_path"]   = test_df["ID"].apply(lambda x: os.path.join(test_img_dir, x, "rgb.png"))
test_df["depth_path"] = test_df["ID"].apply(lambda x: os.path.join(test_depth_dir, x, "depth_raw.png"))

# Filter valid pairs
test_df = test_df[test_df["img_path"].apply(check_file) & test_df["depth_path"].apply(check_file)]

# Compute volume estimates for each test sample
test_df["Volume"] = test_df["depth_path"].apply(estimate_food_volume).astype("float32")

test_ds = build_test_dataset(test_df, batch_size=batch_size)

best_model = tf.keras.models.load_model("best_inception.keras", compile=False)

preds = best_model.predict(test_ds, verbose=1)
test_df["Value"] = preds.squeeze()
test_df = test_df[['ID', 'Value']]


output_csv = "Inception-Volume.csv"
test_df.to_csv(output_csv, index=False)

3/3 ━━━━━━━━━━━━━━━━━━━━ 1s 143ms/step
